# Traffic Sign Classification with Keras

Keras exists to make coding deep neural networks simpler. To demonstrate just how easy it is, you’re going to use Keras to build a convolutional neural network in a few dozen lines of code.

You’ll be connecting the concepts from the previous lessons to the methods that Keras provides.

## Dataset

The network you'll build with Keras is similar to the example that you can find in Keras’s GitHub repository that builds out a [convolutional neural network for MNIST](https://github.com/fchollet/keras/blob/master/examples/mnist_cnn.py). 

However, instead of using the [MNIST](http://yann.lecun.com/exdb/mnist/) dataset, you're going to use the [German Traffic Sign Recognition Benchmark](http://benchmark.ini.rub.de/?section=gtsrb&subsection=news) dataset that you've used previously.

You can download pickle files with sanitized traffic sign data here.

## Overview

Here are the steps you'll take to build the network:

1. First load the data.
2. Build a feedforward neural network to classify traffic signs.
3. Build a convolutional neural network to classify traffic signs.

Keep an eye on the network’s accuracy over time. Once the accuracy reaches the 98% range, you can be confident that you’ve built and trained an effective model.

## Load the Data

Start by importing the data from the pickle file.

In [1]:
# Import stuff
import pickle
import numpy as np

In [2]:
# TODO: Implement load the data here.

training_file = 'train.p'
testing_file = 'test.p'

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)
    
X_train = train['features']
y_train = train['labels']

X_test = test['features']
y_test = test['labels']

# STOP: Do not change the tests below. Your implementation should pass these tests. 
assert(X_train.shape[0] == y_train.shape[0]), "The number of images is not equal to the number of labels."
assert(X_train.shape[1:] == (32,32,3)), "The dimensions of the images are not 32 x 32 x 3."

## Normalize the data

Now that you've loaded the training data, normalize the input so that it has a mean of 0 and a range between -0.5 and 0.5.

In [3]:
# TODO: Implement data normalization here.
## Normalize: scale from 0 to 1
def normalize(image_list):
    normalized_train = {}
    normalized_train['features'] = []
    max_val = np.max(image_list)
    min_val = np.min(image_list)
    halfway = (max_val - min_val) / 2
    for i in range(len(image_list)):
        normalized_train['features'].append(((image_list[i]) - halfway) / (max_val - min_val))
    return np.asarray(normalized_train['features'])

n_X_train = normalize(X_train)
X_train = n_X_train
n_X_test = normalize(X_test)
X_test = n_X_test



# STOP: Do not change the tests below. Your implementation should pass these tests. 
assert(round(np.mean(X_train)) == 0), "The mean of the input data is: %f" % np.mean(X_train)
assert(np.min(X_train) == -0.5 and np.max(X_train) == 0.5), "The range of the input data is: %.1f to %.1f" % (np.min(X_train), np.max(X_train))

## Build a Two-Layer Feedfoward Network

The code you've written so far is for data processing, not specific to Keras. Here you're going to build Keras-specific code.

Build a two-layer feedforward neural network, with 128 neurons in the fully-connected hidden layer. 

To get started, review the Keras documentation about [models](https://keras.io/models/sequential/) and [layers](https://keras.io/layers/core/).

The Keras example of a [Multi-Layer Perceptron](https://github.com/fchollet/keras/blob/master/examples/mnist_mlp.py) network is similar to what you need to do here. Use that as a guide, but keep in mind that there are a number of differences.

In [4]:
# TODO: Build a two-layer feedforward neural network with Keras here.

# imports
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.optimizers import Adam
from keras.utils import np_utils

# Some info
image_dim = X_train[0].shape
input_dim = image_dim[0] * image_dim[1] * image_dim[2]
nb_classes = len(set(y_train))

# One-hot encode the labels
oh_y_train = np_utils.to_categorical(y_train, nb_classes)
oh_y_test = np_utils.to_categorical(y_test, nb_classes)

# Reshape inputs
len_X_train = len(X_train)
len_X_test = len(X_test)
print(len_X_test)

# commenting out the reshape for the CNN
# X_train = X_train.reshape(len_X_train, input_dim)
# X_test = X_test.reshape(len_X_test, input_dim)


# Some params
batch_size = 128
nb_epoch = 2

# Layer params
n_neurons_1 = 128
dropout_1 = 1      # change this to implement dropout!

# Actual NN itself in Keras
model = Sequential()
model.add(Dense(n_neurons_1, input_shape=(input_dim,), name="hidden1"))
model.add(Activation('relu'))
model.add(Dropout(dropout_1))
model.add(Dense(nb_classes))
model.add(Activation('softmax', name="output"))

model.summary() #hey what does this do?


# STOP: Do not change the tests below. Your implementation should pass these tests.
# assert(model.get_layer(name="hidden1").input_shape == (None, 32*32*3)), "The input shape is: %s" % model.get_layer(name="hidden1").input_shape
# assert(model.get_layer(name="output").output_shape == (None, 43)), "The output shape is: %s" % model.get_layer(name="output").output_shape 

assert(model.get_layer(name="hidden1").input_shape == (None, 32*32*3)), "The input shape is: {}".format(model.get_layer(name="hidden1").input_shape)
assert(model.get_layer(name="output").output_shape == (None, 43)), "The output shape is: {}".format(model.get_layer(name="output").output_shape) 

Using TensorFlow backend.


12630
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
hidden1 (Dense)                  (None, 128)           393344      dense_input_1[0][0]              
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 128)           0           hidden1[0][0]                    
____________________________________________________________________________________________________
dropout_1 (Dropout)              (None, 128)           0           activation_1[0][0]               
____________________________________________________________________________________________________
dense_1 (Dense)                  (None, 43)            5547        dropout_1[0][0]                  
_____________________________________________________________________________________

## Train the Network
Compile and train the network for 2 epochs. [Use the `adam` optimizer, with `categorical_crossentropy` loss.](https://keras.io/models/sequential/)

Hint 1: In order to use categorical cross entropy, you will need to [one-hot encode the labels](https://github.com/fchollet/keras/blob/master/keras/utils/np_utils.py). [done]

Hint 2: In order to pass the input images to the fully-connected hidden layer, you will need to [reshape the input](https://github.com/fchollet/keras/blob/master/examples/mnist_mlp.py).

Hint 3: Keras's `.fit()` method returns a `History.history` object, which the tests below use. Save that to a variable named `history`. [done]

In [5]:
# TODO: Compile and train the model here.

#model.compile(loss='categorical_crossentropy',
#             optimizer=Adam(),
#             metrics=['accuracy']) # where does 'accuracy' get defined?

#history = model.fit(X_train, oh_y_train,
#                   batch_size = batch_size,
#                   nb_epoch=nb_epoch,
#                   verbose=1,
#                   validation_data=(X_test, oh_y_test))
#score = model.evaluate(X_test, oh_y_test, verbose=0)
#print('Test score:', score[0])
#print('Test accuracy:', score[1])

# STOP: Do not change the tests below. Your implementation should pass these tests.
#assert(history.history['acc'][0] > 0.5), "The training accuracy was: %.3f" % history.history['acc']

## Validate the Network
Split the training data into a training and validation set.

Measure the [validation accuracy](https://keras.io/models/sequential/) of the network after two training epochs.

Hint: [Use the `train_test_split()` method](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) from scikit-learn.

In [7]:
# TODO: Split some of the training data into a validation dataset.
# TODO: Compile and train the model to measure validation accuracy.

from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.25)

oh_y_val = np_utils.to_categorical(y_val, nb_classes)
oh_y_train = np_utils.to_categorical(y_train, nb_classes)

model.compile(loss='categorical_crossentropy',
             optimizer=Adam(),
             metrics=['accuracy']) # where does 'accuracy' get defined?

history = model.fit(X_train, oh_y_train,
                   batch_size = batch_size,
                   nb_epoch=nb_epoch,
                   verbose=1,
                   validation_data=(X_val, oh_y_val))
score = model.evaluate(X_test, oh_y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(round(X_train.shape[0] / float(X_val.shape[0])) == 3), "The training set is %.3f times larger than the validation set." % X_train.shape[0] / float(X_val.shape[0])
assert(history.history['val_acc'][0] > 0.6), "The validation accuracy is: %.3f" % history.history['val_acc'][0]

Train on 29406 samples, validate on 9803 samples
Epoch 1/2
29406/29406 [==============================] - 51s - loss: 1.1668 - acc: 0.6983 - val_loss: 0.4670 - val_acc: 0.8695
Epoch 2/2
29406/29406 [==============================] - 49s - loss: 0.2916 - acc: 0.9286 - val_loss: 0.2376 - val_acc: 0.9411
Test score: 0.610083789516
Test accuracy: 0.844893111677


**Validation Accuracy**: 0.8255

## Congratulations
You've built a feedforward neural network in Keras!

Don't stop here! Next, you'll add a convolutional layer to drive.py.

## Convolutions
Build a new network, similar to your existing network. Before the hidden layer, add a 3x3 [convolutional layer](https://keras.io/layers/convolutional/#convolution2d) with 32 filters and valid padding.

Then compile and train the network.

Hint 1: The Keras example of a [convolutional neural network](https://github.com/fchollet/keras/blob/master/examples/mnist_cnn.py) for MNIST would be a good example to review.

Hint 2: Now that the first layer of the network is a convolutional layer, you no longer need to reshape the input images before passing them to the network. You might need to reload your training data to recover the original shape.

Hint 3: Add a [`Flatten()` layer](https://keras.io/layers/core/#flatten) between the convolutional layer and the fully-connected hidden layer.

In [10]:
# TODO: Re-construct the network and add a convolutional layer before the first fully-connected layer.
# TODO: Compile and train the model.

from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D

# some info
img_rows = image_dim[0]
img_cols = image_dim[1]
input_shape = image_dim

# some params

# conv kernel size
kernel_size = (3, 3)
nb_filters = 32

model = Sequential()
model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1],
                        border_mode='valid',
                        input_shape=input_shape))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(n_neurons_1, input_shape=(input_dim,), name="hidden1"))
model.add(Activation('relu'))
model.add(Dropout(dropout_1))
model.add(Dense(nb_classes))
model.add(Activation('softmax', name="output"))

model.summary() #hey what does this do?

model.compile(loss='categorical_crossentropy',
             optimizer=Adam(),
             metrics=['accuracy']) # where does 'accuracy' get defined?

history = model.fit(X_train, oh_y_train,
                   batch_size = batch_size,
                   nb_epoch=nb_epoch,
                   verbose=1,
                   validation_data=(X_val, oh_y_val))
score = model.evaluate(X_test, oh_y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])


# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(history.history['val_acc'][0] > 0.9), "The validation accuracy is: %.3f" % history.history['val_acc'][0]

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_3 (Convolution2D)  (None, 30, 30, 32)    896         convolution2d_input_3[0][0]      
____________________________________________________________________________________________________
activation_6 (Activation)        (None, 30, 30, 32)    0           convolution2d_3[0][0]            
____________________________________________________________________________________________________
flatten_3 (Flatten)              (None, 28800)         0           activation_6[0][0]               
____________________________________________________________________________________________________
hidden1 (Dense)                  (None, 128)           3686528     flatten_3[0][0]                  
___________________________________________________________________________________________

AssertionError: The validation accuracy is: 0.887

**Validation Accuracy**: 0.9248

## Pooling
Re-construct your network and add a 2x2 [pooling layer](https://keras.io/layers/pooling/#maxpooling2d) immediately following your convolutional layer.

Then compile and train the network.

In [11]:
# TODO: Re-construct the network and add a pooling layer after the convolutional layer.
# TODO: Compile and train the model.

from keras.layers import MaxPooling2D

# some params

# conv kernel size
kernel_size = (3, 3)
nb_filters = 32

pool_size = (2, 2)

model = Sequential()
model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1],
                        border_mode='valid',
                        input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=pool_size))
model.add(Flatten())
model.add(Dense(n_neurons_1, input_shape=(input_dim,), name="hidden1"))
model.add(Activation('relu'))
model.add(Dropout(dropout_1))
model.add(Dense(nb_classes))
model.add(Activation('softmax', name="output"))

model.summary() #hey what does this do?

model.compile(loss='categorical_crossentropy',
             optimizer=Adam(),
             metrics=['accuracy']) # where does 'accuracy' get defined?

history = model.fit(X_train, oh_y_train,
                   batch_size = batch_size,
                   nb_epoch=nb_epoch,
                   verbose=1,
                   validation_data=(X_val, oh_y_val))
score = model.evaluate(X_test, oh_y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])


# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(history.history['val_acc'][0] > 0.9), "The validation accuracy is: %.3f" % history.history['val_acc'][0]

# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(history.history['val_acc'][0] > 0.9), "The validation accuracy is: %.3f" % history.history['val_acc'][0]

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_4 (Convolution2D)  (None, 30, 30, 32)    896         convolution2d_input_4[0][0]      
____________________________________________________________________________________________________
activation_8 (Activation)        (None, 30, 30, 32)    0           convolution2d_4[0][0]            
____________________________________________________________________________________________________
maxpooling2d_1 (MaxPooling2D)    (None, 15, 15, 32)    0           activation_8[0][0]               
____________________________________________________________________________________________________
flatten_4 (Flatten)              (None, 7200)          0           maxpooling2d_1[0][0]             
___________________________________________________________________________________________

AssertionError: The validation accuracy is: 0.801

**Validation Accuracy**: 0.9054

## Dropout
Re-construct your network and add [dropout](https://keras.io/layers/core/#dropout) after the pooling layer. Set the dropout rate to 50%.

In [14]:
# TODO: Re-construct the network and add dropout after the pooling layer.
# TODO: Compile and train the model.

# some params
nb_epoch = 10

# conv kernel size
kernel_size = (3, 3)
nb_filters = 32

pool_size = (2, 2)
p_dropout=0.5


model = Sequential()
model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1],
                        border_mode='valid',
                        input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=pool_size))
model.add(Dropout(p_dropout))
model.add(Flatten())
model.add(Dense(n_neurons_1, input_shape=(input_dim,), name="hidden1"))
model.add(Activation('relu'))
model.add(Dropout(dropout_1))
model.add(Dense(nb_classes))
model.add(Activation('softmax', name="output"))

model.summary() #hey what does this do?

model.compile(loss='categorical_crossentropy',
             optimizer=Adam(),
             metrics=['accuracy']) # where does 'accuracy' get defined?

history = model.fit(X_train, oh_y_train,
                   batch_size = batch_size,
                   nb_epoch=nb_epoch,
                   verbose=1,
                   validation_data=(X_val, oh_y_val))
score = model.evaluate(X_test, oh_y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(history.history['val_acc'][0] > 0.9), "The validation accuracy is: %.3f" % history.history['val_acc'][0]

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_7 (Convolution2D)  (None, 30, 30, 32)    896         convolution2d_input_7[0][0]      
____________________________________________________________________________________________________
activation_14 (Activation)       (None, 30, 30, 32)    0           convolution2d_7[0][0]            
____________________________________________________________________________________________________
maxpooling2d_4 (MaxPooling2D)    (None, 15, 15, 32)    0           activation_14[0][0]              
____________________________________________________________________________________________________
dropout_10 (Dropout)             (None, 15, 15, 32)    0           maxpooling2d_4[0][0]             
___________________________________________________________________________________________

AssertionError: The validation accuracy is: 0.798

In [ ]:
**Validation Accuracy**: 0.9163

## Optimization
Congratulations! You've built a neural network with convolutions, pooling, dropout, and fully-connected layers, all in just a few lines of code.

Have fun with the model and see how well you can do! Add more layers, or regularization, or different padding, or batches, or more training epochs.

What is the best validation accuracy you can achieve?

**Best Validation Accuracy:** 97.88% with 10 epochs

## Testing
Once you've picked out your best model, it's time to test it.

Load up the test data and use the [`evaluate()` method](https://keras.io/models/model/#evaluate) to see how well it does.

Hint 1: After you load your test data, don't forget to normalize the input and one-hot encode the output, so it matches the training data. [wait this is already done for us]

Hint 2: The `evaluate()` method should return an array of numbers. Use the `metrics_names()` method to get the labels.

In [19]:
with open('./test.p', mode='rb') as f:
    test = pickle.load(f)
    
X_test = test['features']
y_test = test['labels']
X_test = X_test.astype('float32')
X_test /= 255
X_test -= 0.5
Y_test = np_utils.to_categorical(y_test, 43)

output = model.evaluate(X_test, Y_test)
print(output)
print(model.metrics_names)

12630/12630 [==============================] - 7s     
[0.47971021296221966, 0.90174188443997128]
['loss', 'acc']


**Test Accuracy:** 90.7%

## Summary
Keras is a great tool to use if you want to quickly build a neural network and evaluate performance.